In [1]:
using Gridap
using Gridap.Algebra;
using Gridap.FESpaces
using Gridap.ReferenceFEs
using Gridap.Arrays
using Gridap.Geometry
using Gridap.Fields
using Gridap.CellData

using GridapGmsh

using LinearAlgebra

using FillArrays
using SparseArrays

In [2]:
try
    using Gmsh
catch
    using gmsh
end

In [3]:
ri = 19.1e-3;   # Cable inner radius
ro = 37.5e-3;   # Insulation outer radius
rj = 46.5e-3;   # Cable outer radius

lc1 = 5e-3;     # Mesh density at outside of cable
lc2 = 1e-3;   # Mesh density at the conductor edge
lc3 = 30e-3;

x1 =     0; y1 = 0;
x2 = -4*rj; y2 = 0;
x3 =  4*rj; y3 = 0;

In [4]:
function gmsh_add_circle(mid, radius, lc)
    geo = gmsh.model.geo;
    
    # Corner points
    p1 = geo.addPoint(mid[1] + radius, mid[2], 0, lc);
    p2 = geo.addPoint(mid[1] - radius, mid[2], 0, lc);
    p3 = geo.addPoint(mid[1], mid[2], 0, 1);
    points = [p1, p2];
    
    # Lines
    l1 = geo.addCircleArc(p1, p3, p2);
    l2 = geo.addCircleArc(p2, p3, p1);
    lines = [l1, l2];
    
    # Curve loop
    loop = geo.addCurveLoop(lines);
    
    return loop, lines, points;
end

gmsh_add_circle (generic function with 1 method)

In [5]:
gmsh.initialize()

In [6]:
gmsh.model.add("cable_flat")
geo = gmsh.model.geo;

## Domain
domain_lp, domain_lines, _ = gmsh_add_circle([0, 0], 10*rj, lc3);

## Cables
con1_lp, _, _ = gmsh_add_circle([x1, y1], ri, lc2);
ins1_lp, _, _ = gmsh_add_circle([x1, y1], ro, lc1);
jac1_lp, _, _ = gmsh_add_circle([x1, y1], rj, lc1);

con2_lp, _, _ = gmsh_add_circle([x2, y2], ri, lc2);
ins2_lp, _, _ = gmsh_add_circle([x2, y2], ro, lc1);
jac2_lp, _, _ = gmsh_add_circle([x2, y2], rj, lc1);

con3_lp, _, _ = gmsh_add_circle([x3, y3], ri, lc2);
ins3_lp, _, _ = gmsh_add_circle([x3, y3], ro, lc1);
jac3_lp, _, _ = gmsh_add_circle([x3, y3], rj, lc1);

## Plane surfaces
domain_surf = geo.addPlaneSurface([domain_lp, jac1_lp, jac2_lp, jac3_lp])

jac1_surf = geo.addPlaneSurface([jac1_lp, ins1_lp])
jac2_surf = geo.addPlaneSurface([jac2_lp, ins2_lp])
jac3_surf = geo.addPlaneSurface([jac3_lp, ins3_lp])

ins1_surf = geo.addPlaneSurface([ins1_lp, con1_lp])
ins2_surf = geo.addPlaneSurface([ins2_lp, con2_lp])
ins3_surf = geo.addPlaneSurface([ins3_lp, con3_lp])

con1_surf = geo.addPlaneSurface([con1_lp])
con2_surf = geo.addPlaneSurface([con2_lp])
con3_surf = geo.addPlaneSurface([con3_lp])

geo.synchronize();

## Physical domains
boundary = geo.addPhysicalGroup(1, domain_lines)
domain   = geo.addPhysicalGroup(2, [domain_surf])

con1 = geo.addPhysicalGroup(2, [con1_surf])
con2 = geo.addPhysicalGroup(2, [con2_surf])
con3 = geo.addPhysicalGroup(2, [con3_surf])

insulator = geo.addPhysicalGroup(2, [ins1_surf, ins2_surf, ins3_surf])
jacket    = geo.addPhysicalGroup(2, [jac1_surf, jac2_surf, jac3_surf])

gmsh.model.setPhysicalName(1, boundary, "Boundary")
gmsh.model.setPhysicalName(2, domain, "Air")
gmsh.model.setPhysicalName(2, con1, "Conductor1")
gmsh.model.setPhysicalName(2, con2, "Conductor2")
gmsh.model.setPhysicalName(2, con3, "Conductor3")
gmsh.model.setPhysicalName(2, insulator, "Insulator")
gmsh.model.setPhysicalName(2, jacket, "Jacket")

geo.synchronize()

gmsh.model.mesh.generate(2);

gmsh.write("geo/hv_cable_flat.msh")

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Circle)
Info    : [ 10%] Meshing curve 2 (Circle)
Info    : [ 10%] Meshing curve 3 (Circle)
Info    : [ 20%] Meshing curve 4 (Circle)
Info    : [ 20%] Meshing curve 5 (Circle)
Info    : [ 30%] Meshing curve 6 (Circle)
Info    : [ 30%] Meshing curve 7 (Circle)
Info    : [ 40%] Meshing curve 8 (Circle)
Info    : [ 40%] Meshing curve 9 (Circle)
Info    : [ 50%] Meshing curve 10 (Circle)
Info    : [ 50%] Meshing curve 11 (Circle)
Info    : [ 60%] Meshing curve 12 (Circle)
Info    : [ 60%] Meshing curve 13 (Circle)
Info    : [ 70%] Meshing curve 14 (Circle)
Info    : [ 70%] Meshing curve 15 (Circle)
Info    : [ 80%] Meshing curve 16 (Circle)
Info    : [ 80%] Meshing curve 17 (Circle)
Info    : [ 90%] Meshing curve 18 (Circle)
Info    : [ 90%] Meshing curve 19 (Circle)
Info    : [100%] Meshing curve 20 (Circle)
Info    : Done meshing 1D (Wall 0.00797796s, CPU 0.015625s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Plane

In [7]:
gmsh.fltk.run()

-------------------------------------------------------
Version       : 4.10.2
License       : GNU General Public License
Build OS      : Windows64-sdk
Build date    : 19700101
Build host    : amdci7
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen NoSocklenT ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.6.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------



(process:37068): GLib-GIO-WARNING **: 16:26:36.592: Unexpectedly, UWP app `SiemensPLMSoftware.JT2GoDesktop_14.1.1.0_x64__qtbmxjdagz8xc' (AUMId `SiemensPLMSoftware.JT2GoDesktop_qtbmxjdagz8xc!JT2GoDesktop') supports 18 extensions but has no verbs


# Problem Setup

In [8]:
I = 1000;             # Current in the cable [A]

mu0 = 4*pi*1e-7;      # Permeability of vacuum
f = 50;               # Frequency of the time-harmonic current
omega = 2*pi*f;      

sigma_cond = 3.69e7;  # Conductivity of aluminium

# Gridap Setup

In [9]:
# Define norm of a complex vector for Gridap to use on the CellField which results from Bh = Ã¢ÂˆÂ‡(uh)
norm_complex(a::VectorValue{2, ComplexF64}) = sqrt(norm(a[1])^2 + norm(a[2])^2)
norm_complex(a::CellField) = Operation(norm_complex)(a)

norm_complex (generic function with 2 methods)

# Gridap

In [10]:
# Define Lagrangian reference element
order = 2;
reffe = ReferenceFE(lagrangian, Float64, order)

# Load mesh
model = GmshDiscreteModel("geo/hv_cable_flat.msh")
Th = Triangulation(model)
Qh = CellQuadrature(Th, 2*order)

# Construct Lagrangian test space with dirichlet condition on the right boundary node
Vh = TestFESpace(model, reffe; conformity = :H1, dirichlet_tags = ["Boundary"], vector_type = Vector{ComplexF64})
Uh = TrialFESpace(Vh, [0])

Info    : Reading 'geo/hv_cable_flat.msh'...
Info    : 60 entities
Info    : 10922 nodes
Info    : 21842 elements
Info    : Done reading 'geo/hv_cable_flat.msh'


TrialFESpace()

In [11]:
labels = get_face_labeling(model)
dimension = num_cell_dims(model)
tags = get_face_tag(labels, dimension)

# Construct a cell field consisting of the physical domain tags
τ = CellField(tags, Th)

const tag_con1 = get_tag_from_name(labels, "Conductor1");
const tag_con2 = get_tag_from_name(labels, "Conductor2");
const tag_con3 = get_tag_from_name(labels, "Conductor3");

function fsource(tag)
    return 0.0;
end

function fsigma(tag)
    if tag == tag_con1 || tag == tag_con2 || tag == tag_con3
        return sigma_cond;
    else
        return 0.0;
    end
end

function fmur(tag)
    return 1.0;
end

function fnu(tag)
    return 1 / (mu0 * fmur(tag))
end

fnu (generic function with 1 method)

In [12]:
dv = get_fe_basis(Vh);
du = get_trial_fe_basis(Uh);

# Calculate contributions of the cells to the linear system
cellvals_A = ∫( (fnu ∘ τ) ⋅ ∇(du) ⋅ ∇(dv) - 1im * omega * (fsigma ∘ τ) ⋅ du ⋅ dv )*Qh;
cellvals_f = ∫( (fsource ∘ τ) ⋅ dv )*Qh;

# Assemble the linear system
assem = SparseMatrixAssembler(Uh, Vh)
σk = get_cell_dof_ids(Uh);

rs = ([cellvals_f], [σk]);
b = allocate_vector(assem, rs);
assemble_vector!(b, assem, rs);

rs = ([cellvals_A], [σk], [σk]);
A = allocate_matrix(assem, rs);
assemble_matrix!(A, assem, rs);

In [13]:
# Extend the linear system with an additional row and column per circuit equation
N  = num_free_dofs(Uh);
nc = 3;

C = Matrix(UniformScaling(1), nc, nc);
f2 = [
    I * exp(1im * 0), 
    I * exp(1im * 2pi/3),
    I * exp(-1im * 2pi/3)
];

# Calculate the cell contributions to the extra vectors
Th_con1 = Triangulation(model, tags = "Conductor1"); Qh_con1 = CellQuadrature(Th_con1, 2*order)
Th_con2 = Triangulation(model, tags = "Conductor2"); Qh_con2 = CellQuadrature(Th_con2, 2*order)
Th_con3 = Triangulation(model, tags = "Conductor3"); Qh_con3 = CellQuadrature(Th_con3, 2*order)

cellvals_b1T = ∫( (1/(pi*ri^2)) ⋅ dv )*Qh_con1;
cellvals_b2T = ∫( (1/(pi*ri^2)) ⋅ dv )*Qh_con2;
cellvals_b3T = ∫( (1/(pi*ri^2)) ⋅ dv )*Qh_con3;

cellvals_b1  = ∫( (1im * omega * (fsigma ∘ τ)) ⋅ dv )*Qh_con1;
cellvals_b2  = ∫( (1im * omega * (fsigma ∘ τ)) ⋅ dv )*Qh_con2;
cellvals_b3  = ∫( (1im * omega * (fsigma ∘ τ)) ⋅ dv )*Qh_con3;

# Assemble the circuit coupling equations into two extra vectors
σk_con1 = get_cell_dof_ids(Uh, Th_con1);
σk_con2 = get_cell_dof_ids(Uh, Th_con2);
σk_con3 = get_cell_dof_ids(Uh, Th_con3);

rs = ([cellvals_b1T], [σk_con1])
b1T = allocate_vector(assem, rs)
assemble_vector!(b1T, assem, rs);

rs = ([cellvals_b2T], [σk_con2])
b2T = allocate_vector(assem, rs)
assemble_vector!(b2T, assem, rs);

rs = ([cellvals_b3T], [σk_con3])
b3T = allocate_vector(assem, rs)
assemble_vector!(b3T, assem, rs);

rs = ([cellvals_b1], [σk_con1])
b1 = allocate_vector(assem, rs)
assemble_vector!(b1, assem, rs);

rs = ([cellvals_b2], [σk_con2])
b2 = allocate_vector(assem, rs)
assemble_vector!(b2, assem, rs);

rs = ([cellvals_b3], [σk_con3])
b3 = allocate_vector(assem, rs)
assemble_vector!(b3, assem, rs);

In [14]:
# Create the modified system \tilde{A} = [A BT; B C] and \tilde{b} = [b1; b2]
At = [A [b1T b2T b3T]; transpose([b1 b2 b3]) C];
bt = [b; f2];

# Solve the modified linear system and package the solution in a FEFunction
u = At \ bt;
uh = FEFunction(Uh, u);

In [15]:
# Post-processing for magnetic field and current density
Bh = norm_complex(∇(uh));

function fsource(tag)
    if tag == tag_con1
        return u[end-2] / (pi * ri^2);
    elseif tag == tag_con2
        return u[end-1] / (pi * ri^2);
    elseif tag == tag_con3
        return u[end-0] / (pi * ri^2);
    else
        return 0.0 + 0.0im;
    end
end

J0    = fsource ∘ τ; # Source current density
Jeddy = 1im ⋅ omega ⋅ (fsigma ∘ τ) ⋅ mean(uh); # Eddy current density

In [16]:
writevtk(Th, "images/hvcable_gridap/flat", cellfields=[
    "Az"    => abs(uh), 
    "normB" => Bh,
    "normJ" => abs(J0 + Jeddy)
])

(["images/hvcable_gridap/flat.vtu"],)

In [17]:
dΩc1 = Measure(Th_con1, 2*order);
dΩc2 = Measure(Th_con2, 2*order);
dΩc3 = Measure(Th_con3, 2*order);

Rac1 = ∫( abs(J0 + Jeddy) * abs(J0 + Jeddy) / (fsigma ∘ τ) )*dΩc1;
Rac2 = ∫( abs(J0 + Jeddy) * abs(J0 + Jeddy) / (fsigma ∘ τ) )*dΩc2;
Rac3 = ∫( abs(J0 + Jeddy) * abs(J0 + Jeddy) / (fsigma ∘ τ) )*dΩc3;

I1   = ∑(∫( J0 + Jeddy )dΩc1)
I2   = ∑(∫( J0 + Jeddy )dΩc2)
I3   = ∑(∫( J0 + Jeddy )dΩc3)

Rac1 = real(∑( Rac1 ) / abs(I1)^2)
Rac2 = real(∑( Rac2 ) / abs(I2)^2)
Rac3 = real(∑( Rac3 ) / abs(I3)^2)

2.690997413808122e-5

In [18]:
print("Rac1 = $(Rac1*1e6) uR/m\nRac2 = $(Rac2*1e6) uR/m\nRac3 = $(Rac3*1e6) uR/m\n")

Rac1 = 27.29597891102474 uR/m
Rac2 = 26.904576740497973 uR/m
Rac3 = 26.90997413808122 uR/m


In [27]:
Rdc = 1 / (sigma_cond * pi * ri^2);

# Skin effect
ks = 1
xs = sqrt(8*pi*f / Rdc * 1e-7 * ks)
ys = xs^4 / (192 + 0.8 * xs^4);
if(xs < 0 || xs > 2.8) error("Analytical formula not valid for given xs = {xs}") end

# Proximity effect
kp = 1
xp = sqrt(8*pi*f / Rdc * 1e-7 * kp)
ap = xp^4 / (192 + 0.8 * xp^4);

yp1 = ap * (2*ri / (4*rj))^2 * (0.312 * (ri / rj)^2 + 1.18 / (ap + 0.27));
yp2 = ap * (2*ri / sqrt(4*8*rj^2))^2 * (0.312 * (2*ri / sqrt(4*8*rj^2))^2 + 1.18 / (ap + 0.27));

if(xp < 0 || xp > 2.8) error("Analytical formula not valid for given xs = {xs}") end

Rac1_a = Rdc * (1 + ys + yp1)
Rac2_a = Rdc * (1 + ys + yp2)
print("Analytical Rac1 = $(Rac1_a * 1e6) uR/m\n")
print("Analytical Rac2 = $(Rac2_a * 1e6) uR/m\n")
print("Analytical Rac3 = $(Rac2_a * 1e6) uR/m\n")

Analytical Rac1 = 27.150588456763764 uR/m
Analytical Rac2 = 26.951272176741796 uR/m
Analytical Rac3 = 26.951272176741796 uR/m


![Flux Density](images/hvcable_gridap/normB_flat.png)

![Current Density](images/hvcable_gridap/normJ_flat.png)